In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
# Carregar o DataFrame
df = pd.read_excel('C:/Users/Patyc/OneDrive/Desktop/Dissertation/Data/Skin_clean and treated/Skin_clean.xlsx')

## Converter a data para datetime e ordenar de forma crescente
df['Screening_date'] = pd.to_datetime(df['Screening_date'])
df = df.sort_values(by='Screening_date')

In [3]:
# 2. Criar uma coluna para representar a semana e o ano
df['Week'] = df['Screening_date'].dt.strftime('%Y-%U')

# 3. Contar o número de screenings por semana
df = df.groupby('Week').size().reset_index(name='total_screenings_per_week')

df

,Week,total_screenings_per_week
0,2022-09,92
1,2022-11,62
2,2022-12,90
3,2022-14,139
4,2022-16,173
...,...,...
70,2024-11,168
71,2024-12,198
72,2024-13,242
73,2024-15,186


In [6]:
# 4. Realizar o teste ADF para verificar a estacionaridade
result = adfuller(df['total_screenings_per_week'])

# Extrair e imprimir os resultados
adf_statistic = result[0]
p_value = result[1]
used_lag = result[2]
n_observations = result[3]
critical_values = result[4]

print('\nResultados do teste ADF:')
print('ADF Statistic:', adf_statistic)
print('p-value:', p_value)
print('Used Lag:', used_lag)
print('Number of Observations:', n_observations)
print('Critical Values:')
for key, value in critical_values.items():
    print(f'   {key}: {value}')
    
# Interpretação dos resultados
if p_value < 0.05:
    print("Rejeitamos a hipótese nula: a série é estacionária.")
else:
    print("Não rejeitamos a hipótese nula: a série não é estacionária.")



Resultados do teste ADF:
ADF Statistic: -2.5089171433396316
p-value: 0.11331973560316377
Used Lag: 2
Number of Observations: 72
Critical Values:
   1%: -3.524624466842421
   5%: -2.9026070739026064
   10%: -2.5886785262345677
Não rejeitamos a hipótese nula: a série não é estacionária.


In [4]:
from statsmodels.tsa.stattools import adfuller

# Realizar o teste ADF na coluna 'total_screenings_per_week'
adf_result = adfuller(df['total_screenings_per_week'].dropna())

# Exibir os resultados do teste
print('ADF Statistic:', adf_result[0])
print('p-value:', adf_result[1])
print('Critical Values:')
for key, value in adf_result[4].items():
    print(f'   {key}: {value}')

ADF Statistic: -2.5089171433396316
p-value: 0.11331973560316377
Critical Values:
   1%: -3.524624466842421
   5%: -2.9026070739026064
   10%: -2.5886785262345677


In [95]:
df['total_screenings_diff'] = df['total_screenings_per_week'].diff().dropna()
adf_result_diff = adfuller(df['total_screenings_diff'].dropna())


In [97]:
# Realizar o teste ADF na coluna 'total_screenings_per_week'
adf_result = adfuller(df['total_screenings_diff'].dropna())

# Exibir os resultados do teste
print('ADF Statistic:', adf_result[0])
print('p-value:', adf_result[1])
print('Critical Values:')
for key, value in adf_result[4].items():
    print(f'   {key}: {value}')

ADF Statistic: -10.191121840419315
p-value: 6.318315042041935e-18
Critical Values:
   1%: -3.524624466842421
   5%: -2.9026070739026064
   10%: -2.5886785262345677


In [98]:
df.rename(columns={'total_screenings_diff': 'Total_screenings'}, inplace=True)

df

,Week,total_screenings_per_week,Total_screenings
0,2022-09,92,NaN
1,2022-11,62,-30.0
2,2022-12,90,28.0
3,2022-14,139,49.0
4,2022-16,173,34.0
...,...,...,...
70,2024-11,168,-240.0
71,2024-12,198,30.0
72,2024-13,242,44.0
73,2024-15,186,-56.0


In [99]:
# Eliminar a coluna 'Week' e 'total_screenings_per_week'
df = df.drop(columns=['total_screenings_per_week'])

df

,Week,Total_screenings
0,2022-09,NaN
1,2022-11,-30.0
2,2022-12,28.0
3,2022-14,49.0
4,2022-16,34.0
...,...,...
70,2024-11,-240.0
71,2024-12,30.0
72,2024-13,44.0
73,2024-15,-56.0


In [100]:
# Remover linhas onde Total_screenings contém NaN
df = df.dropna(subset=['Total_screenings'])

# Exibir o DataFrame para confirmar a remoção
df

,Week,Total_screenings
1,2022-11,-30.0
2,2022-12,28.0
3,2022-14,49.0
4,2022-16,34.0
5,2022-19,-170.0
...,...,...
70,2024-11,-240.0
71,2024-12,30.0
72,2024-13,44.0
73,2024-15,-56.0


# LR MODEL

In [101]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [102]:
# Supõe-se que o dataframe está carregado na variável df
df['Week'] = np.arange(len(df))  # Transformando as semanas em um índice numérico para o modelo

X = df[['Week']]  # Variável independente
y = df['Total_screenings']  # Variável dependente



C:\Users\Patyc\AppData\Local\Temp\ipykernel_16756\1909281005.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Week'] = np.arange(len(df))  # Transformando as semanas em um índice numérico para o modelo


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [103]:
df

,Week,Total_screenings
1,0,-30.0
2,1,28.0
3,2,49.0
4,3,34.0
5,4,-170.0
...,...,...
70,69,-240.0
71,70,30.0
72,71,44.0
73,72,-56.0


In [104]:
# Define a semana de corte
split_week = '2024-09'

# Certifique-se de que a coluna 'Week' é do tipo string
df['Week'] = df['Week'].astype(str)

# Agora, divida os dados com base na semana de corte
train = df[df['Week'] < split_week]
test = df[df['Week'] >= split_week]

# Separando X e y para cada conjunto
X_train = train.drop(columns=['Total_screenings', 'Week'])
y_train = train['Total_screenings']

X_test = test.drop(columns=['Total_screenings', 'Week'])
y_test = test['Total_screenings']

C:\Users\Patyc\AppData\Local\Temp\ipykernel_16756\754540653.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Week'] = df['Week'].astype(str)


In [105]:
df['Week']

1      0
2      1
3      2
4      3
5      4
      ..
70    69
71    70
72    71
73    72
74    73
Name: Week, Length: 74, dtype: object

In [106]:
# Adicionar uma constante aos dados
X_train_sm = sm.add_constant(X_train)
X_test_sm = X_test.copy()  # Certifique-se de que não está sobrescrevendo X_test original
X_test_sm.insert(0, 'const', 1.0)

# print(X_train_sm.head())
# print(X_test_sm.head())

# Ajustar o modelo
model_sm = sm.OLS(y_train, X_train_sm).fit()

# Obter o resumo do modelo
print(model_sm.summary())

# Fazer previsões no conjunto de teste
y_pred = model_sm.predict(X_test_sm)

# Calcular e imprimir as métricas de avaliação
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
me = np.mean(y_test - y_pred)
mav = np.mean(np.abs(y_test))
mpv = np.mean(np.abs((y_test - y_pred) / y_test))
rme = np.mean((y_test - y_pred) / y_test)
rmae = np.mean(np.abs(y_test - y_pred) / np.abs(y_test))

                            OLS Regression Results                            
Dep. Variable:       Total_screenings   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Fri, 16 Aug 2024   Prob (F-statistic):                nan
Time:                        16:25:52   Log-Likelihood:                -75.352
No. Observations:                  14   AIC:                             152.7
Df Residuals:                      13   BIC:                             153.3
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.7857     14.598     -0.328      0.7

c:\Users\Patyc\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=14 observations were given.
  return hypotest_fun_in(*args, **kwds)


In [107]:
print("Modelo: LR")
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')
print(f'ME: {me}')
print(f'MAV: {mav}')
print(f'MPV: {mpv}')
print(f'RME: {rme}')
print(f'RMAE: {rmae}')

# Modelo: LR shuffle
# MAE: 99.7851431911163
# MSE: 15949.548258569848
# RMSE: 126.29152092903881
# R2: -0.0015887323702037381
# ME: 4.9245080849405785
# MAV: 99.66666666666667
# MPV: 1.0710098169084572
# RME: 1.0710098169084572
# RMAE: 1.0710098169084572

# Modelo: LR split date
# MAE: 73.56190476190477
# MSE: 10809.384013605442
# RMSE: 103.96818750755176
# R2: -0.0051412835039283955
# ME: 7.435714285714285
# MAV: 73.21666666666667
# MPV: 1.1333033737837097
# RME: 0.8344938499741856
# RMAE: 1.1333033737837097

Modelo: LR
MAE: 73.56190476190477
MSE: 10809.384013605442
RMSE: 103.96818750755176
R2: -0.0051412835039283955
ME: 7.435714285714285
MAV: 73.21666666666667
MPV: 1.1333033737837097
RME: 0.8344938499741856
RMAE: 1.1333033737837097


# LSTM MODEL

In [70]:
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import LSTM, Dense # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from sklearn.preprocessing import MinMaxScaler

In [71]:
# Supondo que seu dataframe seja chamado df e a coluna de interesse seja 'Total_screenings'
data = df[['Total_screenings']].values

# Normalizando os dados para ficar entre 0 e 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Função para criar sequências para o LSTM
def create_sequences(data, seq_length):
    x = []
    y = []
    for i in range(seq_length, len(data)):
        x.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(x), np.array(y)

# Definir o comprimento da sequência (número de passos de tempo usados para prever o próximo valor)
seq_length = 10
x, y = create_sequences(scaled_data, seq_length)

# Remodelando os dados para a forma [amostras, tempo, características]
x = np.reshape(x, (x.shape[0], x.shape[1], 1))

# Dividindo os dados em treino e teste
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(x_train, y_train, epochs=20, batch_size=32)

c:\Users\Patyc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.2310
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1581 
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1150 
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0655 
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0301 
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0253 
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0390 
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0395 
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0320 
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0243 
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0223 
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0288 
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0275
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0284 
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0313 
Epoch 16/20
2/2 ━━━━━━━━━━━━━━━━━━

In [72]:
# Fazer previsões
y_pred = model.predict(x_test)

# Desnormalizar as previsões para o formato original
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calcular e imprimir as métricas de avaliação
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
me = np.mean(y_test - y_pred)
mav = np.mean(np.abs(y_test))
mpv = np.mean(np.abs((y_test - y_pred) / y_test))
rme = np.mean((y_test - y_pred) / y_test)
rmae = np.mean(np.abs(y_test - y_pred) / np.abs(y_test))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 523ms/step


In [73]:
print("Modelo: LSTM")
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')
print(f'ME: {me}')
print(f'MAV: {mav}')
print(f'MPV: {mpv}')
print(f'RME: {rme}')
print(f'RMAE: {rmae}')

# Modelo: LSTM shuffle
# MAE: 104.81201641376202
# MSE: 17056.0495501771
# RMSE: 130.59881144243656
# R2: -0.06992907183085695
# ME: -32.243974538949814
# MAV: 96.84615384615384
# MPV: 4.957557399315164
# RME: 4.915422248699178
# RMAE: 4.957557399315164

Modelo: LSTM
MAE: 104.81201641376202
MSE: 17056.0495501771
RMSE: 130.59881144243656
R2: -0.06992907183085695
ME: -32.243974538949814
MAV: 96.84615384615384
MPV: 4.957557399315164
RME: 4.915422248699178
RMAE: 4.957557399315164


In [85]:
# Supõe-se que o dataframe está carregado na variável df
df['Week'] = np.arange(len(df))  # Transformando as semanas em um índice numérico para o modelo

X = df[['Week']]  # Variável independente
y = df['Total_screenings']  # Variável dependente

C:\Users\Patyc\AppData\Local\Temp\ipykernel_16756\3143327643.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Week'] = np.arange(len(df))  # Transformando as semanas em um índice numérico para o modelo


In [87]:
# Define a semana de corte
split_week = '2024-09'

# Certifique-se de que a coluna 'Week' é do tipo string
df['Week'] = df['Week'].astype(str)

# Agora, divida os dados com base na semana de corte
train = df[df['Week'] < split_week]
test = df[df['Week'] >= split_week]

# Separando X e y para cada conjunto
X_train = train.drop(columns=['Total_screenings', 'Week'])
y_train = train['Total_screenings']

X_test = test.drop(columns=['Total_screenings', 'Week'])
y_test = test['Total_screenings']

C:\Users\Patyc\AppData\Local\Temp\ipykernel_16756\754540653.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Week'] = df['Week'].astype(str)


In [88]:
# Supondo que seu dataframe seja chamado df e a coluna de interesse seja 'Total_screenings'
data = df[['Total_screenings']].values

# Normalizando os dados para ficar entre 0 e 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Função para criar sequências para o LSTM
def create_sequences(data, seq_length):
    x = []
    y = []
    for i in range(seq_length, len(data)):
        x.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(x), np.array(y)

# Definir o comprimento da sequência (número de passos de tempo usados para prever o próximo valor)
seq_length = 10
x, y = create_sequences(scaled_data, seq_length)

# Remodelando os dados para a forma [amostras, tempo, características]
x = np.reshape(x, (x.shape[0], x.shape[1], 1))

# Dividindo os dados em treino e teste
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(x_train, y_train, epochs=20, batch_size=32)

Epoch 1/20


c:\Users\Patyc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.2671
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1757 
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1122 
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0600 
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0280 
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0258 
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0378 
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0413 
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0323 
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0227 
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0268 
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0246 
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0299
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0311 
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0248
Epoch 16/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st

In [89]:
# Fazer previsões
y_pred = model.predict(x_test)

# Desnormalizar as previsões para o formato original
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calcular e imprimir as métricas de avaliação
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
me = np.mean(y_test - y_pred)
mav = np.mean(np.abs(y_test))
mpv = np.mean(np.abs((y_test - y_pred) / y_test))
rme = np.mean((y_test - y_pred) / y_test)
rmae = np.mean(np.abs(y_test - y_pred) / np.abs(y_test))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step


In [90]:
print("Modelo: LSTM")
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')
print(f'ME: {me}')
print(f'MAV: {mav}')
print(f'MPV: {mpv}')
print(f'RME: {rme}')
print(f'RMAE: {rmae}')

# Modelo: LSTM split
# MAE: 103.74068128145657
# MSE: 16983.456576509074
# RMSE: 130.3205915291558
# R2: -0.06537530146864134
# ME: -28.67636519211988
# MAV: 96.84615384615384
# MPV: 4.417209739304278
# RME: 4.398065299607012
# RMAE: 4.417209739304278

Modelo: LSTM
MAE: 103.74068128145657
MSE: 16983.456576509074
RMSE: 130.3205915291558
R2: -0.06537530146864134
ME: -28.67636519211988
MAV: 96.84615384615384
MPV: 4.417209739304278
RME: 4.398065299607012
RMAE: 4.417209739304278
